# Modulation Scheme Example

In [1]:
import numpy as np
import datetime, os, sys, pickle
import math
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python import debug as tf_debug

In [5]:
# --------------------------- Communication Scheme Definitions -----------------------------
noOfSymbols     = 4
noOfChannels    = 2     # 2 for I-Q
EbN0_dB         = 7.0 # EbN0 at which the network is trained
# --------------------------------- Training Parameters ------------------------------------
# Generate training data
minibatch_size  =  10
nEpochs         =  10
eval_interval   =  1
# --------------------------------- Testing Parameters -------------------------------------
test_data_size  = 100
test_data       = to_categorical( np.random.choice(noOfSymbols,test_data_size), noOfSymbols )
symbols         = to_categorical( np.arange(0,noOfSymbols), noOfSymbols )
# ------------------------------------------------------------------------------------------
symbolLength    = (np.log2(noOfSymbols))
R   = symbolLength/noOfChannels     # Communication Rate
# Calculate value of Eb/N0 from dB
EbN0    = 10 ** (EbN0_dB/10)

In [ ]:
# ---------------------------------- Auto encoder model ------------------------------------
with tf.name_scope( "Input_symbol" ):
    tx_in = tf.placeholder( dtype = tf.float32, shape = (None,noOfSymbols), name = "tx_symbol" )

with tf.name_scope( "Transmitter"):
    with tf.name_scope( "TX_1" ):
        l1_wgt  = tf.Variable( tf.random_normal([noOfSymbols,noOfSymbols],
                                                stddev=np.sqrt(3.0/(noOfSymbols+noOfSymbols))), 
                                name = "l1_wgt" )
        l1_bias = tf.Variable( tf.zeros([noOfSymbols]), name = "l2_bias" )
        l1_out  = tf.nn.relu( tf.matmul(tx_in,l1_wgt) + l1_bias )
    
    with tf.name_scope( "TX_2" ):
        l2_wgt  = tf.Variable( tf.random_normal([noOfSymbols,noOfChannels],
                                                stddev=np.sqrt(3.0/(noOfSymbols+noOfChannels))), 
                                name = "l2_wgt" )
        l2_bias = tf.Variable( tf.zeros([noOfChannels]), name = "l2_bias" )
        l2_out  = tf.matmul(l1_out,l2_wgt) + l2_bias

    with tf.name_scope( "TX_normalization" ):
        l3_out  = np.sqrt(0.5*noOfChannels) * tf.multiply( 
                                l2_out,
                                tf.rsqrt( tf.maximum(
                                    tf.reduce_sum( tf.square(l2_out), axis = 1, keepdims = True ),
                                    1e-12 )
                                ),
                                name = "l3_out"
                            )
    
    tx_out  = tf.identity( l3_out, name = "tx_out" )
    
with tf.name_scope( "Channel" ):
    ch_noise = tf.random_normal( shape = tf.shape(tx_out), 
                                 mean = 0.0, stddev = np.sqrt(1/(2*R*EbN0)),
                                 dtype = tf.float32 )
    ch_out  = tf.add( tx_out, ch_noise, name = "ch_out" )

with tf.name_scope( "Reciever" ):
    rx_in   = tf.identity( ch_out, name = "rx_in" )
    with tf.name_scope( "RX_1" ):
        # rx1_out     = tf.layers.dense( inputs = rx_in, units = noOfSymbols, activation = tf.nn.relu )
        rx1_wgt     = tf.Variable( tf.random_normal([noOfChannels,noOfSymbols],
                                                    stddev=np.sqrt(3.0/(noOfChannels+noOfSymbols))), 
                                    name = "rx1_wgt" )
        rx1_bias    = tf.Variable( tf.zeros([noOfSymbols]), name = "rx1_bias")
        rx1_out     = tf.nn.relu( tf.matmul(rx_in,rx1_wgt) + rx1_bias )

    with tf.name_scope( "RX_2" ):
        # rx2_out     = tf.layers.dense( inputs = rx1_out, units = noOfSymbols, activation = tf.nn.softmax )
        rx2_wgt     = tf.Variable( tf.random_normal([noOfSymbols,noOfSymbols],
                                                    stddev=np.sqrt(3.0/(noOfSymbols+noOfSymbols))), 
                                    name = "rx2_wgt" )
        rx2_bias    = tf.Variable( tf.zeros([noOfSymbols]), name = "rx2_bias" )
        rx2_out     = tf.nn.softmax( tf.matmul(rx1_out,rx2_wgt) + rx2_bias, name = "rx2_out" )

    rx_out  = tf.identity( rx2_out, name = "rx_out" )

with tf.name_scope( "Evaluation" ):
    loss_metric = tf.reduce_mean( -tf.reduce_sum(
                                        tx_in*tf.log(rx_out+1e-12), # Adding a small number to avoid xEnt explosion
                                        reduction_indices=1) )
    # loss_metric = tf.reduce_mean(tf.pow(tx_in - rx_out, 2))
    # cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(p), reduction_indices=1))
    
# Make train graph
with tf.name_scope( "Optimizers" ):
    # Get the parameters to train at Tx and Rx separately
    rx_params       = tf.get_collection( tf.GraphKeys.TRAINABLE_VARIABLES, "Reciever" )
    tx_params       = tf.get_collection( tf.GraphKeys.TRAINABLE_VARIABLES, "Transmitter" )

    # Define Optimizers for Tx and Rx
    rx_optimizer    = tf.train.AdamOptimizer( learning_rate = 0.01, beta1 = 0.99, beta2 = 0.999 )
    # rx_optimizer    = SimultaneousPerturbationOptimizer( a = 0.10, c = 0.10, alpha = 0.90, gamma = 0.50 )
    # tx_optimizer    = tf.train.AdamOptimizer( learning_rate = 0.01, beta1 = 0.99, beta2 = 0.999 )
    tx_optimizer    = SimultaneousPerturbationOptimizer( a = 0.05, c = 0.10, alpha = 0.90, gamma = 0.30 )
        
    # Attach the objective
    rx_objective    = rx_optimizer.minimize( loss_metric, var_list = rx_params )
    tx_objective    = tx_optimizer.minimize( loss_metric, var_list = tx_params )